In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "8g") \
    .appName('vb-app') \
    .getOrCreate()

In [4]:
path = "../data/raw/wls_day-01"
df = spark.read.json(path)

In [5]:
# df.show()

In [7]:
df.columns

['AuthenticationPackage',
 'Destination',
 'DomainName',
 'EventID',
 'FailureReason',
 'LogHost',
 'LogonID',
 'LogonType',
 'LogonTypeDescription',
 'ParentProcessID',
 'ParentProcessName',
 'ProcessID',
 'ProcessName',
 'ServiceName',
 'Source',
 'Status',
 'SubjectDomainName',
 'SubjectLogonID',
 'SubjectUserName',
 'Time',
 'UserName']

In [10]:
# df.groupBy('EventID').count().show()

+-------+--------+
|EventID|   count|
+-------+--------+
|   4625|  335425|
|   4608|    1607|
|   4802|    5148|
|   4672| 4388232|
|   4770|      85|
|   4801|   14899|
|   1100|    1637|
|   4648|  895145|
|   4647|    3060|
|   4624|14110061|
|   4768|  696025|
|   4769| 1762893|
|   4688|14034308|
|   4634|13826193|
|   4689|    1111|
|   4800|   15917|
|   4803|    4833|
|   4609|       9|
|   4776| 5505769|
|   4774|    6794|
+-------+--------+



In [11]:
df.groupBy('UserName').count().show()

+-----------+-----+
|   UserName|count|
+-----------+-----+
|Comp334881$| 2327|
|Comp193344$| 1753|
|Comp626532$| 1696|
|Comp423822$| 1581|
|Comp071603$| 1716|
|Comp194392$| 1541|
|Comp079982$| 2467|
|Comp102246$| 1822|
|Comp924592$| 1855|
|Comp332130$| 5977|
|Comp629929$| 1149|
|Comp989334$| 1676|
|Comp851861$| 2293|
|Comp854936$| 1737|
|Comp282520$| 3297|
|Comp349824$| 2129|
|Comp156468$| 1536|
|Comp528070$| 1714|
|Comp811288$| 1865|
|Comp925148$| 1705|
+-----------+-----+
only showing top 20 rows



In [12]:
# df.groupBy('LogonType').count().show()

+---------+--------+
|LogonType|   count|
+---------+--------+
|        0|    1616|
|        7|   73211|
|     null|27337472|
|        9|   24850|
|        5|  191974|
|       10|    2271|
|        3|27642694|
|        8|   89498|
|       11|     589|
|        2|  240504|
|        4|    4472|
+---------+--------+



In [13]:
# df.groupBy('Source').count().show()

+----------+-----+
|    Source|count|
+----------+-----+
|Comp095019|  351|
|Comp361193|30688|
|Comp515833| 3941|
|Comp129210|  368|
|Comp265496| 1469|
|Comp793733| 5792|
|Comp213727|  774|
|Comp403467|  796|
|Comp852067|  360|
|Comp449379| 1039|
|Comp495040|  809|
|Comp893211|  857|
|Comp237460|  665|
|Comp795960| 1428|
|Comp044445|  571|
|Comp886769|  872|
|Comp189990| 1668|
|Comp879962|  458|
|Comp721320|  843|
|Comp507421| 1300|
+----------+-----+
only showing top 20 rows



## Convert time into timestamp

In [8]:
# df.groupBy('Time').count().show()

+----+-----+
|Time|count|
+----+-----+
|  26|  524|
|  29|  476|
| 474|  459|
| 964|  655|
|1677|  691|
|1697|  676|
|1806|  544|
|1950|  492|
|2040|  707|
|2214|  413|
|2250|  772|
|2453|  573|
|2509|  605|
|2529|  591|
|2927|  442|
|3091|  642|
|3506|  457|
|3764|  386|
|4590|  459|
|4823|  654|
+----+-----+
only showing top 20 rows



In [ ]:
# from pyspark.sql.functions import from_unixtime

# df = df.withColumn('timestamp', from_unixtime((df.Time.cast('bigint')/1000)).cast('timestamp'))


In [22]:
df.take(10)

[Row(AuthenticationPackage=None, Destination=None, DomainName='Domain001', EventID=4688, FailureReason=None, LogHost='Comp607982', LogonID='0x3e7', LogonType=None, LogonTypeDescription=None, ParentProcessID='0x2ac', ParentProcessName='services', ProcessID='0x1418', ProcessName='svchost.exe', ServiceName=None, Source=None, Status=None, SubjectDomainName=None, SubjectLogonID=None, SubjectUserName=None, Time=1, UserName='Comp607982$', timestamp=datetime.datetime(1969, 12, 31, 19, 0)),
 Row(AuthenticationPackage=None, Destination=None, DomainName='Domain001', EventID=4688, FailureReason=None, LogHost='Comp991643', LogonID='0x3e7', LogonType=None, LogonTypeDescription=None, ParentProcessID='0x334', ParentProcessName='services', ProcessID='0xc0c', ProcessName='rundll32.exe', ServiceName=None, Source=None, Status=None, SubjectDomainName=None, SubjectLogonID=None, SubjectUserName=None, Time=1, UserName='Comp991643$', timestamp=datetime.datetime(1969, 12, 31, 19, 0)),
 Row(AuthenticationPackage

In [23]:
# df.groupBy('timestamp').count().show()

+-------------------+------+
|          timestamp| count|
+-------------------+------+
|1969-12-31 19:00:11|489183|
|1969-12-31 19:00:15|456370|
|1969-12-31 19:00:00|579335|
|1969-12-31 19:00:35|885784|
|1969-12-31 19:00:09|517308|
|1969-12-31 19:00:06|521435|
|1969-12-31 19:00:49|674520|
|1969-12-31 19:01:04|656014|
|1969-12-31 19:00:26|780890|
|1969-12-31 19:00:38|817478|
|1969-12-31 19:00:03|503628|
|1969-12-31 19:00:12|487315|
|1969-12-31 19:00:13|481071|
|1969-12-31 19:00:34|947169|
|1969-12-31 19:00:42|740721|
|1969-12-31 19:00:58|653614|
|1969-12-31 19:01:12|623624|
|1969-12-31 19:00:41|741600|
|1969-12-31 19:01:19|610040|
|1969-12-31 19:01:23|632211|
+-------------------+------+
only showing top 20 rows



## Create Window Functions (Fix Usernames)

### Total events per user in an (last) hour window interval  
Ref: https://stackoverflow.com/questions/45806194/pyspark-rolling-average-using-timeseries-data

In [5]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
new_df = df.select("Time", "UserName", "EventID")
w = (Window.partitionBy('UserName').orderBy(F.col("Time").cast('long')).rangeBetween(-3600, Window.currentRow))
new_df = new_df.withColumn('total_events', F.count("Time").over(w))

In [6]:
new_df.orderBy("Time").take(10)

[Row(Time=1, UserName='Comp026768$', rolling_sum=1),
 Row(Time=1, UserName='Comp213870$', rolling_sum=2),
 Row(Time=1, UserName='Comp213870$', rolling_sum=2),
 Row(Time=1, UserName='Comp339863$', rolling_sum=2),
 Row(Time=1, UserName='Comp299420$', rolling_sum=2),
 Row(Time=1, UserName='Comp299420$', rolling_sum=2),
 Row(Time=1, UserName='Comp339863$', rolling_sum=2),
 Row(Time=1, UserName='Comp502166$', rolling_sum=1),
 Row(Time=1, UserName='Comp506838$', rolling_sum=1),
 Row(Time=1, UserName='Comp683515$', rolling_sum=3)]

## Paritition By UserName and Event

In [22]:
w = (Window.partitionBy('UserName', 'EventID').orderBy(F.col("Time").cast('long')).rangeBetween(-3600, Window.currentRow))
new_df = new_df.withColumn('total_per_event', F.count("Time").over(w))

### Total number of Event 4624 per user in an hour window interval  (Successful Logon)


In [35]:
new_df.filter((new_df.UserName == 'Comp502166$') & (new_df.EventID == 4624)).show()

+-----+-----------+-------+---------------+
| Time|   UserName|EventID|total_per_event|
+-----+-----------+-------+---------------+
| 1859|Comp502166$|   4672|              1|
| 8829|Comp502166$|   4672|              1|
|14231|Comp502166$|   4672|              1|
|17894|Comp502166$|   4672|              1|
|21016|Comp502166$|   4672|              2|
|28038|Comp502166$|   4672|              1|
|33680|Comp502166$|   4672|              1|
|39684|Comp502166$|   4672|              1|
|45626|Comp502166$|   4672|              1|
|46694|Comp502166$|   4672|              2|
|51208|Comp502166$|   4672|              1|
|56970|Comp502166$|   4672|              1|
|62974|Comp502166$|   4672|              1|
|68924|Comp502166$|   4672|              1|
|75286|Comp502166$|   4672|              1|
|75494|Comp502166$|   4672|              2|
|80868|Comp502166$|   4672|              1|
+-----+-----------+-------+---------------+



### Total number of Event 4625 per user in an hour window interval  (Failed Logon)

### Total number of Event 4627 per user in an hour window interval  (Group Membership)


### Total number of Event 4648 per user in an hour window interval  (A logon was attempted using explicit credentials)

### Total number of Event 4658 per user in an hour window interval  (handle to an object was closed)

### Total number of Event 4661 per user in an hour window interval  (handle to an object was requsted)

### Total number of Event 4672 per user in an hour window interval  (Special privileges assigned to new logon)

### Total number of Event 4768 per user in an hour window interval  (Kerberos Authentication)

### Total number of Event 4697 per user in an hour window interval  (A service was installed in the system)

### Total number of Event 4698 per user in an hour window interval  (A scheduled task was created)

### Total number of Event 4779 per user in an hour window interval  (A security-enabled local group membership was enumerated)

### Total number of Event 5140 per user in an hour window interval  (A network share object was accessed)

### Total number of Event 5145 per user in an hour window interval  (A network share object was checked to see whether client can be granted desired access)

### Total number of Event 5158 per user in an hour window interval  (The Windows Filtering Platform has permitted a bind to a local port)

## Concatenate datasets 